# 일단 돌기는 돈다.
- accuracy 왜 계속 1.000이니? : trainable True로 바꿔 주고 다시 해 보기
- 일단 모델 레이어 층만 보고.

# 모르겠는 거?!
- leakyrelu가 activation function이 아니라 층인가?
- activation 층?
- batchnormalization 정확히 모르겠다.

In [1]:
# module import
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Flatten, Reshape
from tensorflow.keras.layers import LeakyReLU ## 이거가 층인가?
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

In [3]:
# 데이터 로드: X_train 데이터만 사용
(X_train, _), (_, _) = mnist.load_data()

In [4]:
# CNN 모델 입력 형태로 데이터 바꾸기
print(f"<before> Train Data: {X_train.shape}")
image_size = X_train.shape[1]
n_channel = 1 # 흑백 이미지
X_train = np.reshape(X_train, [-1, image_size, image_size, n_channel])
print(f"<after> Train Data: {X_train.shape}")

<before> Train Data: (60000, 28, 28)
<after> Train Data: (60000, 28, 28, 1)


In [5]:
# 이미지 데이터 표준화: 출력 지움.
print("<sample>")
print(X_train[0])
X_train = X_train.astype('float32') / 255.0
print(X_train[0])

<sample>
[[[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]

In [6]:
# latent vector size 설정
# latent_size = int(input('latent size 설정: '))
latent_size = 100

latent size 설정: 100


# Generator

- 초기 레이어 설정: 7 x 7 x 128
    - 평활화한 뒤
    - reshape을 통해 3차원 입력 형태로 바꿔 준다

- 반복문을 통해 layer 쌓는다. strides 설정 다르게 준다. 앞의 두 필터 사이즈(128, 64)에는 strides를 2로, 뒤의 두 필터 사이즈에는 strides를 1로!
- 쌓으면서 내내 똑같아야 하니까 'same' 옵션 주는 듯?


In [7]:
def build_generator(inputs, image_size):
    image_resize = image_size // 4

    # generator 네트워크 파라미터
    kernel_size = 5
    layer_filters = [128, 64, 32, 1]

    # 초기 입력층 설정
    x = Dense(image_resize * image_resize * layer_filters[0])(inputs)
    print(f"평활화 초기 층: {x.shape}")
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)
    print(f"reshape 후 초기 층: {x.shape}")

    # 은닉층
    for filters in layer_filters:
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1

        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)
        
        print(f"이번 층: {x.shape}")
    
    # 마지막 activation 층
    x = Activation('sigmoid')(x)
    g_model = Model(inputs, x, name='generator')

    print("====== Generator 모델 전체 구조 ======")
    print(g_model.summary())

    return g_model

# Discriminator
* 초기 레이어 설정: 그냥 input
* 필터 사이즈는 점점 늘어 난다.
* 첫 3개 레이어 필터에는 stride 2 적용하고, 마지막 필터 레이어만 1로 잡아 준다.
* `LeakyReLU` : https://www.tensorflow.org/api_docs/python/tf/keras/layers/LeakyReLU
* dense(1) 왜 노드 1?


In [8]:
def build_discriminator(inputs):
    kernel_size = 5
    layer_filters = [32, 64, 128, 256]

    # 입력
    x = inputs
    print(f"초기 입력: {x.shape}")

    # 은닉층 쌓기
    for filters in layer_filters:
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2

        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)
        
        print(f"이번 층: {x.shape}")
    
    # activation 층
    # 평활화 후 Dense로 뽑아 내기
    x = Flatten()(x)
    print(f"평활화 후: {x.shape}")
    x = Dense(1)(x)
    print(f"Dense 거친 후: {x.shape}")
    x = Activation('sigmoid')(x)
    print(f"마지막 활성화 후: {x.shape}")

    # 모델 구성
    d_model = Model(inputs, x, name='discriminator')
    print("====== discriminator 모델 전체 구조 ======")
    print(d_model.summary())

    return d_model

# 훈련

* discriminator 학습
    - 1.0, 0.0 라벨 하나씩 생성해서 지도학습 방식으로 훈련.
    - 데이터셋에서 실제 이미지 랜덤하게 추출
    - 가짜 이미지 생성.
    - 실제 이미지와 가짜 이미지를 묶어 하나의 훈련 배치로 생성.
    - 라벨 만드는 것은 이전과 동일한 방식.

* generator(GAN= adversarial) 학습
    - fake image 1배치. 라벨 1 나오도록 학습.
    - discriminator 웨이트는 업데이트하지 않도록 설정.
    - 

* log 모아서 한 번에 출력!

* 모델 만드는 건 나중에 진행한다.
* 논문은 Adam 사용했으나, RMSprop이 discriminator에서는 더 잘 작동하는듯하다.

* discriminator compile metrics accuracy.


* 여기서의 adversarial model = 앞에서의 GAN model인 듯.

    - 왜 lr 반으로 주는가?
    - 모델 자체를 넣는다.


In [16]:
def train(models, X_train, params):

    # GAN 모델 구성 요소
    generator, discriminator, adversarial = models

    # 네트워크 파라미터
    batch_size, latent_size, train_steps, model_names = params

    # 훈련 데이터 개수
    train_size = X_train.shape[0]

    # 훈련
    for i in range(train_steps):
        
        # 랜덤하게 이미지 고르기
        rand_indices = np.random.randint(0, train_size, size=batch_size)
        real_images = X_train[rand_indices]

        # fake image 생성: generator
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        fake_images = generator.predict(noise)

        # 훈련 데이터 1배치 생성
        X = np.concatenate((real_images, fake_images))

        # 실제 이미지 라벨 생성한 후, 반만 가짜 이미지로 바꾸기. 배치 뒤쪽이 가짜 이미지.
        y = np.ones([2*batch_size, 1])
        y[batch_size:, :] = 0.0 # 뒤쪽 라벨은 가짜 이미지이므로 0.0

        # discriminator 학습
        loss, acc = discriminator.train_on_batch(X, y)
        log = "Epoch %d: [D-loss: %.4f, acc: %.4f]" % (i, loss, acc)

        # generator: noise에 대해서 이미지 생성하고 1로 학습
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        y = np.ones([batch_size, 1])
        loss, acc = adversarial.train_on_batch(noise, y)
        
        log = "%s [G-loss: %.4f, acc: %.4f]" % (log, loss, acc) # 로그 한 번에 출력
        print(log)
    
    # generator 훈련 뒤 모델 저장
    generator.save(model_name + ".h5")

# 모델 빌드 및 훈련

In [13]:
def build_and_train_models(load_W=False, train_steps=100):
    model_name = "dcgan_mnist"

   # 네트워크 파라미터 설정
    batch_size = 64
    lr = 2e-4
    decay = 6e-8
    input_shape = (image_size, image_size, 1)


    ### Discriminator Model
    inputs = Input(shape=input_shape, name='Discriminator_Input')
    D = build_discriminator(inputs)
    
    # discriminator: RMSprop optimizer
    print("====== Discriminator ======")
    D_opt = RMSprop(lr=lr, decay=decay)
    D.compile(loss='binary_crossentropy',
              optimizer=D_opt,
              metrics=['accuracy'])
    # print(D.summary())

    # 저장된 모델 있으면 불러 오기
    if load_W:
        D.load_weights(f"{data_path}/dcgan_D.h5")

    ### Generator Model: 학습하지 않음.
    print("====== Discriminator ======")
    input_shape = (latent_size, )
    inputs = Input(shape=input_shape, name='Z_Input')
    G = build_generator(inputs, image_size)
    # print(D.summary())

    # 저장된 모델 있으면 불러 오기
    if load_W:
        G.load_weights(f"{data_path}/dcgan_D.h5")
    
    ### Adversarial Model: generator 모델 학습 + discriminator까지 같이 feed.
    print("====== GAN 네트워크 ======")
    g_opt = RMSprop(lr=lr*0.5, decay=decay*0.5)
    # D.trainable = False # GAN 네트워크 도중 discriminator 업데이트 해제
    GAN = Model(inputs, D(G(inputs)), name=model_name)
    GAN.compile(loss='binary_crossentropy', optimizer=g_opt, metrics=['accuracy'])
    print(GAN.summary())

    # D, GAN 네트워크 학습
    models = (G, D, GAN)
    params = (batch_size, latent_size, train_steps, model_name)
    train(models, X_train, params) ##################################################

    # 모델 저장
    D.save_weights(f"{data_path}/dcgan_D.h5")
    G.save_weights(f"{data_path}/dcgan_G.h5")

# 그림 그리기

In [14]:
def plot_images():
    inputs = Input(shape=(latent_size, ), name='Z_input')
    G = build_generator(inputs, image_size)
    G.load_weights(f"{data_path}/dcgan_G.h5")

    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    images = G.predict(noise_input) # 가짜 이미지 생성
    plt.figure(figsize=(6, 6))
    num_images = images.shape[0]

    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    rows = int(np.sqrt(noise_input.shape[0]))

    for i in range(num_images):
        plt.subplot(rows, rows, i+1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()

# 학습!
- 여기서는 이미 강사님이 학습해 둔 weight를 읽어 오고, 추가로 학습한다.
- 그런데! 에러가 난다. : load weights with 5 layers 9 layers... 대충 뭐 층 안맞는다는 뜻인듯

In [17]:
build_and_train_models(load_W=False, train_steps=100)

초기 입력: (None, 28, 28, 1)
이번 층: (None, 14, 14, 32)
이번 층: (None, 7, 7, 64)
이번 층: (None, 4, 4, 128)
이번 층: (None, 4, 4, 256)
평활화 후: (None, 4096)
Dense 거친 후: (None, 1)
마지막 활성화 후: (None, 1)
====== discriminator 모델 전체 구조 ======
Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Discriminator_Input (InputLa [(None, 28, 28, 1)]       0         
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 32)        832       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          51264     
______________________________

NameError: ignored

In [ ]:
# 이미지 표시
plot_images()

# 테스트

In [ ]:
dir(mnist)
mnist.load_data?

In [ ]:
# def build_generator(inputs, height=n_height, width=n_width):
#     # resize image
#     new_height = height // 4
#     new_width = width // 4

#     # generator 파라미터
#     kernel_size = 5
#     layer_filters = [128, 64, 32, 1]

#     # 초기 층 설정
#     x = Dense(new_height * new_width * layer_filters[0])(inputs)
#     print(f"초기 평활화 층: {x.shape}")
#     x = Reshape(target_shape=(new_height, new_width, layer_filters[0]))(x)
#     print(f"3차원 reshape 후: {x.shape}")

In [ ]:
# 초기 층 만든 뒤
build_generator(X_train[0], 28, 28)